In [12]:
import sqlite3
from langchain_community.vectorstores import FAISS
from tqdm import tqdm
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.chat_models import init_chat_model

In [2]:
db_path = r"c:\classroom\nov7\courses.db"

In [3]:
def get_courses():
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM courses")
    courses = cursor.fetchall()
    conn.close()
    return courses

In [4]:
# Convert a tuple to string 
def get_string(course : tuple) -> str:
    # unpack tuple 
    course_id, title, description, duration, fee, prerequisite = course
    return f"Title: {title}\nDescription: {description}\nFee: {fee}\nDuration: {duration}\nPrerequisite: {prerequisite}"

In [5]:
# Load courses data
courses = get_courses()  # list[tuple]

courses_list = []
for course in courses:
    courses_list.append(get_string(course))

In [6]:
print(courses_list)

['Title: Python\nDescription: This covers all concepts of Python Language and important libraries\nFee: 7500\nDuration: 36\nPrerequisite: None', 'Title: DataScience\nDescription: This teaches how to build machine learing models\nFee: 10000\nDuration: 36\nPrerequisite: Python', 'Title: Generative AI\nDescription: This teaches how to get the best from LLM using Prompt Engineering, augment them using RAG and generate code using AI pair programming\nFee: 10000\nDuration: 18\nPrerequisite: Python', 'Title: Full-stack Web Development\nDescription: This teaches how to develop a full-stack web application using Node and Express for backend and React for frontend\nFee: 10000\nDuration: 36\nPrerequisite: Programming Knowledge']


In [7]:
embeddings_model = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001")
vectorstore = FAISS.from_texts(courses_list, embeddings_model)

In [8]:
question = "Which course covers RAG"
matching_results = vectorstore.similarity_search(question, k=2)

In [9]:
context = ""
for match in matching_results:
    context += match.page_content + "\n\n"

In [10]:
prompt = f"""
You are a helpful assistant. You will be provided with a context and a question. Your task is to answer the question based on the context.
Context : 
{context}

Question: {question}
"""

In [11]:
llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")
response = llm.invoke(prompt)
print(response.content)

The course that covers RAG is **Generative AI**.
